In [0]:
simpleData = [("James","Sales","NY",90000,34,10000),
    ("Michael","Sales","NY",86000,56,20000),
    ("Robert","Sales","CA",81000,30,23000),
    ("Maria","Finance","CA",90000,24,23000),
    ("Raman","Finance","CA",99000,40,24000),
    ("Scott","Finance","NY",83000,36,19000),
    ("Jen","Finance","NY",79000,53,15000),
    ("Jeff","Marketing","CA",80000,25,18000),
    ("Kumar","Marketing","NY",91000,50,21000)
  ]

schema = ["employee_name","department","state","salary","age","bonus"]
df = spark.createDataFrame(data=simpleData, schema = schema)

In [0]:
df.show()

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|        James|     Sales|   NY| 90000| 34|10000|
|      Michael|     Sales|   NY| 86000| 56|20000|
|       Robert|     Sales|   CA| 81000| 30|23000|
|        Maria|   Finance|   CA| 90000| 24|23000|
|        Raman|   Finance|   CA| 99000| 40|24000|
|        Scott|   Finance|   NY| 83000| 36|19000|
|          Jen|   Finance|   NY| 79000| 53|15000|
|         Jeff| Marketing|   CA| 80000| 25|18000|
|        Kumar| Marketing|   NY| 91000| 50|21000|
+-------------+----------+-----+------+---+-----+



In [0]:
## Rank the employee based on salary in each department
from pyspark.sql.window import Window
from pyspark.sql.functions import desc,rank

rank_spec=Window.partitionBy("department").orderBy(desc("salary"))
df.withColumn("Rank",rank().over(rank_spec)).show()


+-------------+----------+-----+------+---+-----+----+
|employee_name|department|state|salary|age|bonus|Rank|
+-------------+----------+-----+------+---+-----+----+
|        Raman|   Finance|   CA| 99000| 40|24000|   1|
|        Maria|   Finance|   CA| 90000| 24|23000|   2|
|        Scott|   Finance|   NY| 83000| 36|19000|   3|
|          Jen|   Finance|   NY| 79000| 53|15000|   4|
|        Kumar| Marketing|   NY| 91000| 50|21000|   1|
|         Jeff| Marketing|   CA| 80000| 25|18000|   2|
|        James|     Sales|   NY| 90000| 34|10000|   1|
|      Michael|     Sales|   NY| 86000| 56|20000|   2|
|       Robert|     Sales|   CA| 81000| 30|23000|   3|
+-------------+----------+-----+------+---+-----+----+



In [0]:
df.withColumn("Rank1",rank().over(Window.partitionBy("department").orderBy(desc("salary")))).show()

+-------------+----------+-----+------+---+-----+-----+
|employee_name|department|state|salary|age|bonus|Rank1|
+-------------+----------+-----+------+---+-----+-----+
|        Raman|   Finance|   CA| 99000| 40|24000|    1|
|        Maria|   Finance|   CA| 90000| 24|23000|    2|
|        Scott|   Finance|   NY| 83000| 36|19000|    3|
|          Jen|   Finance|   NY| 79000| 53|15000|    4|
|        Kumar| Marketing|   NY| 91000| 50|21000|    1|
|         Jeff| Marketing|   CA| 80000| 25|18000|    2|
|        James|     Sales|   NY| 90000| 34|10000|    1|
|      Michael|     Sales|   NY| 86000| 56|20000|    2|
|       Robert|     Sales|   CA| 81000| 30|23000|    3|
+-------------+----------+-----+------+---+-----+-----+



In [0]:
## Lead and Lag 
from pyspark.sql.functions import lead,lag
df.withColumn("prev_emp_sal",lag("salary").over(rank_spec))\
  .withColumn("next_emp_sal",lead("salary").over(rank_spec)).show()


+-------------+----------+-----+------+---+-----+------------+------------+
|employee_name|department|state|salary|age|bonus|prev_emp_sal|next_emp_sal|
+-------------+----------+-----+------+---+-----+------------+------------+
|        Raman|   Finance|   CA| 99000| 40|24000|        null|       90000|
|        Maria|   Finance|   CA| 90000| 24|23000|       99000|       83000|
|        Scott|   Finance|   NY| 83000| 36|19000|       90000|       79000|
|          Jen|   Finance|   NY| 79000| 53|15000|       83000|        null|
|        Kumar| Marketing|   NY| 91000| 50|21000|        null|       80000|
|         Jeff| Marketing|   CA| 80000| 25|18000|       91000|        null|
|        James|     Sales|   NY| 90000| 34|10000|        null|       86000|
|      Michael|     Sales|   NY| 86000| 56|20000|       90000|       81000|
|       Robert|     Sales|   CA| 81000| 30|23000|       86000|        null|
+-------------+----------+-----+------+---+-----+------------+------------+

